In [1]:
%load_ext autoreload

In [19]:
%autoreload
import sys
sys.path.insert(0, '../')

In [20]:
from src.utilities.mluar_utils import *

In [21]:
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset, Dataset, load_from_disk
import numpy as np
from einops import rearrange, reduce, repeat
import torch
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import pyplot as plt
import math
import pandas as pd
import pickle as pkl

pd.set_option('display.max_colwidth', None)


In [22]:
MULTI_LUAR_PATH =  "/mnt/swordfish-pool2/milad/multi-luar-reddit-model/"
LUAR_PATH =  "/mnt/swordfish-pool2/nikhil/LUAR/pretrained_weights/LUAR-MUD/"

In [6]:
# Load models
multiluar_model = AutoModel.from_pretrained(MULTI_LUAR_PATH, trust_remote_code=True)
luar_model = AutoModel.from_pretrained(LUAR_PATH, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("rrivera1849/LUAR-MUD")

In [7]:
# Load data
data_path = '/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_perGenre-HRS2.1/TA2/hrs_06-27-24_english_perGenre-HRS2.1/data/hrs_06-27-24_english_perGenre-HRS2.1_TA2_input'
ground_truth_path = '/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_perGenre-HRS2.1/TA2/hrs_06-27-24_english_perGenre-HRS2.1/groundtruth/hrs_06-27-24_english_perGenre-HRS2.1_TA2'
hiatus_data, _, _ = load_aa_data(data_path, ground_truth_path)

Loading:  /mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_perGenre-HRS2.1/TA2/hrs_06-27-24_english_perGenre-HRS2.1/data/hrs_06-27-24_english_perGenre-HRS2.1_TA2_input


In [8]:
# keep authors with only more than one text
authors_with_multiple_texts = [x[0] for x in hiatus_data.authorID.value_counts().to_dict().items() if x[1] > 1]
hiatus_data = hiatus_data[hiatus_data.authorID.isin(authors_with_multiple_texts)]

In [9]:
hiatus_data = hiatus_data.sample(2000)

In [10]:
hiatus_data_texts = hiatus_data.fullText.tolist()
labels = hiatus_data['authorID'].tolist()

# Compute embeddings
max_seq_length = 736
hiatus_data_embeddings = get_luar_embeddings(hiatus_data_texts, multiluar_model, tokenizer, max_length=max_seq_length, batch_size=1, is_multi_luar=True)

In [11]:
#np.max([len(x.split()) for x in hiatus_data.fullText.tolist()])

### Experiment Design

- For each layer, we find pairs of ground-truth texts written by the same author where the corresponding layer's embedding scored them high compared to other layers
- For each layer, we take a sample of these pairs of texts and prompt ChatGPT to find which linguistic level they are similar

#### Step 1:

In [12]:
# Compute Multi-luar similarity matrices at every layer and average of the layers
muti_luar_layers_sims = [compute_similarities(hiatus_data_embeddings, hiatus_data_embeddings, layer=i) for i in range(7)]
#muti_luar_layers_sims.append(compute_similarities(hiatus_data_embeddings, hiatus_data_embeddings, layer=None))
muti_luar_layers_sims = np.stack(muti_luar_layers_sims)

In [23]:
# Compute significant pairs to layers
layer_to_sig_pairs = {layer: extract_sig_pairs_for_layer(hiatus_data_texts, muti_luar_layers_sims, labels, layer) for layer in range(7)}

In [24]:
layer_x_pairs = []
for layer in [0, 1, 2, 3, 4, 5, 6]:
    print(layer, len(layer_to_sig_pairs[layer]))
    sample_of_pairs = layer_to_sig_pairs[layer][:10]
    layer_x_pairs += [{'text-1': x[0], 'text-2': x[1], 'z-score': x[2], 'layer-sim': x[3], 'layer': layer} for x in sample_of_pairs]
layer_x_pairs_df = pd.DataFrame(layer_x_pairs)

0 606
1 0
2 1
3 2
4 0
5 0
6 5


In [27]:
layer_x_pairs_df[layer_x_pairs_df.layer >= 0].head()

,text-1,text-2,z-score,layer-sim,layer
0,"Honmein Resistance Bands\n\nTake your home workout up many levels with this 3-pack of resistance bands. Each band offers a different level of resistance (low, medium, high) so that you can gradually work your way up to the most effective exercises. Perfect for squats, lunges, donkey kicks, and tricep dips. The three bands are all made in a thick fabric that is durable through the toughest workouts and never rolls up! This set comes in a simple carrying bag so that you can take your workout easily with you to the park, beach, vacation, or even the office. This simple and affordable set is the easiest way to stay toned!","Cra-Z-Art Jigsaw Puzzle Glue with Applicator\n\nAfter spending days or weeks putting together all those pieces of your jigsaw puzzle, what better way to appreciate your work than by turning into a piece of art! Seal in your memories of working on your puzzle with this jigsaw puzzle glue. Simply swipe a thin layer on top of your puzzle using the included applicator, let it set, and your artwork is set! Most puzzles will hold with one layer of glue, but you can put two layers for added protection. And with a 133ml jar, you will have enough to seal in multiple puzzles! The glue does not smell and is non-toxic. It dries clear and it simple to clean off from any accidents. This small bottle can be the key to keeping all your puzzle memories together!","[2.0897844, -1.0721691, -1.1436987, -0.005097844, -0.0016726544, -0.22196682, 0.35483754]","[0.8800262, 0.8021675, 0.80040616, 0.82844263, 0.828527, 0.82310253, 0.83730555]",0
1,"Permanent Vacation\n\n“Permanent Vacation” was shot shortly after <PERSON> dropped out of film school…..and yeah, it looks like it. It´s rough, barebone, and underdeveloped. This kind of aimless, plotless hangout film only works when the characters are interesting and, in my opinion, they´re not. Therefore, the movie becomes a drag despite the short runtime. Nevertheless, I like the rebellious attitude and some of the images. The film works best as an idiosyncratic portrait of New York. Even at this point you could predict that <PERSON> would become a director whose movies are characterized by strong vibes.","Nosferatu the Vampyre\n\nI would simply stay with my wife <PERSON> rather than travelling to Transylvania to meet some weirdo <PERSON> but that´s just me.\n\nGerman expressionism meets New German cinema. “Nosferatu the Vampyre” is my favorite horror remake, my favorite vampire film, and my favorite horror mood piece. The atmosphere and use of light and shadows are simply impeccable. The film is a visceral experience. It´s eerie, haunting, and deeply evocative, and <PERSON> delivers another banger score. It´s also a fascinating portrayal of the total collapse of a patriarchal society because nobody listens to the one woman who knows what´s going on. <PERSON> and <PERSON> are the greatest screen duo in Herzog´s filmography.\n\nHere, here, and here are my previous reviews and here is <PERSON> for his Great Movies selection.\n\n<PERSON> movies ranked","[2.2815864, -0.8527378, 0.16573595, -0.8355943, -0.43191236, -0.38879085, 0.06171658]","[0.9552749, 0.90302163, 0.9200009, 0.90330744, 0.91003734, 0.91075623, 0.9182668]",0
2,"The Hunger\n\nyou belong to me. we belong to each other.\nokay i kind of am obsessed... it doesn't explore its themes of the fear of aging or the loneliness of immortality as deeply as it needs to to maximize its potential, and i wish the intersection of the two was more complicated in <PERSON>, but the smoldering sexuality and the visual balladry are hard not to find bewitching... i think this ought to be at least as much of an 80s vampire classic as the lost boys and fright night, and i honestly prefer this to <PERSON> much more famous top gun, even if its bite doesn't quite match its ambitions. still, i found it very moving.","Hard to Die\n\nnow i remember! <PERSON>'s the guy that was involved in that sorority house massacre a

In [33]:
layer_x_pairs_df.to_pickle('../data/layer_to_pairs_signficance.pkl', 'wb')

#### Step 2:

In [1]:
from datadreamer import DataDreamer
from datadreamer.llms import HFTransformers, ParallelLLM, OpenAI
from datadreamer.steps import DataFromPrompt, ProcessWithPrompt,  HFHubDataSource, DataSource, zipped, concat
from functools import partial
from transformers import QuantoConfig
from datasets import concatenate_datasets, load_dataset
import json


model = OpenAI(model_name="gpt-4", api_key='sk-proj-zTbZNk16Ik1pZnqLn38ZT3BlbkFJImq3pd7widkr7RzsC771') #kathy's lab
# model = HFTransformers(
#                 "meta-llama/Meta-Llama-3-8B-Instruct",
#                 quantization_config=QuantoConfig(weights="int8"),
#                 device=0,
#                 device_map="cuda",
#             )
# model.config.pad_token_id = model.config.eos_token_id

def gen_from_iterable_dataset(iterable_ds):
    yield from iterable_ds

def evaluate_text_similarities(data_path, document_pairs, linguistic_lvl_name, linguistic_lvl_desc):
    instruction = "Given the two Documents below, rate their <linguistic_lvl> on a scale from 1 to 5: Score 1 equals very low similarity and score 5 equals high similarity. First, give reasons for your score and then output the score. The output should be in the following format: {\"reasons\": \"explain your rating\",  \"score\": \"<json integer>\"}"
    instruction = instruction.replace("<linguistic_lvl>", linguistic_lvl_desc)
    
    with DataDreamer(data_path):
        datasource = DataSource('documents', Dataset.from_list(document_pairs))
        datasource = datasource.map(lambda row: {'inputs': 'Document 1:\n {} \n Document 2:\n {}'.format(row['text-1'], row['text-2'])}, auto_progress=False)
        ds_focus_questions = ProcessWithPrompt(
          "{} describe text similarity".format(linguistic_lvl_name),
          inputs={"inputs": datasource.output["inputs"]},
          args={
             "llm": model,
             "n": 1,
             "instruction": instruction
          },
          outputs={"generations": linguistic_lvl_name},
        ).select_columns([linguistic_lvl_name])
        
        zipped_step = zipped(datasource, ds_focus_questions)

        results_iter = zipped_step.output.dataset
        results_ds   = Dataset.from_generator(partial(gen_from_iterable_dataset, results_iter))

        return results_ds

In [8]:
layer_to_sig_pairs = pkl.load(open('../data/layer_to_pairs_signficance.pkl', 'rb'))

In [9]:
layer_x_pairs = []
for layer in [0, 1, 2, 3, 4, 5, 6]:
    print(layer, len(layer_to_sig_pairs[layer]))
    sample_of_pairs = layer_to_sig_pairs[layer][:10]
    layer_x_pairs += [{'text-1': x[0], 'text-2': x[1], 'z-score': x[2], 'layer': layer} for x in sample_of_pairs]

0 2154
1 6
2 20
3 108
4 16
5 56
6 890


In [10]:
len(layer_x_pairs)

66

In [11]:
# Given the two Documents below, rate their <linguistic_lvl> on a scale from 1 to 5: Score 1 equals very low similarity and score 5 equals high similarity. First, give reasons for your score and then output the score in the following output format: {\"reasons\": \"explain your rating\",  \"score\": \"<json integer>\"}
ling_phenomena = {
    'syntax': 'syntactic similarity',
    'semantic': 'semantic similarity',
    'lexical': 'lexical similarity',
    'discourse': 'discourse similarity'
}

In [12]:
results = []
for key, val in ling_phenomena.items():
    results.append(evaluate_text_similarities('./output', layer_x_pairs, key, val))

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./output
[ 🤖 DataDreamer 💤 ] Step 'documents' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' progress: 50 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity (select_columns)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity (select_columns)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'zipped(documents (map), syntax describe text similarity (select_columns))' 

In [13]:
all_results = concatenate_datasets(results)

In [14]:
all_results.save_to_disk('../data/described_similarities_ds')

Saving the dataset (0/1 shards):   0%|          | 0/264 [00:00<?, ? examples/s]

### Analyze layer to lingustic similarity:

In [15]:
all_results = load_from_disk('../data/described_similarities_ds')

In [16]:
all_results_df = all_results.to_pandas()

In [17]:
zscores= all_results_df['z-score'].tolist()

In [26]:
all_results_df[['text-1', 'text-2', 'layer', 'z-score']].head(n=50)

,text-1,text-2,layer,z-score
0,"Anatomy of a Fall\n\ni'm fully aware it’s not a fair comparison, but i’d like to be able to have a foundation of a dead-body-case type work-of-fiction to base my conclusion on how weak or strong the one i’m watching currently is. recently, and in this case, i’ve come to referring to disco elysium, of which strongly elicited throughout my entire lengthy time with it what the evident culmination of anatomy of a fall - the kitchen fight recording, some of the child actors scenes - coaxed. immediately and on a purely cosmetic and surface level, Fall lacks any type of relatively incisive and personally disruptive style. this sort of flatly lit, boringly pictured and blocked, framed with little to no intention movie reminds me of Tar, from a few years ago, and at least for a little while we were happy with this natural Lightroom preset of a color palate. the question remains that if the director moved focus away from the visuals to focus on the performances, and the performances aren’t at all times captivating squarely enough to fill the vacuum, then why are you wasting our time\n\naddendum: <PERSON> states on an interview i read on cinematography world that the original intention was to shoot on 35, and they rather proceeded to shoot hawk v lite anamorphic on alexa then matte out the beautiful academy ratio to widescreen. horrific world we live in","<PERSON>\n\ngoing back and watching these in the original language instead of the goofy english dubs. a lot of people say this is the best one but i'm not really sure. most of them are pretty great. all of them are intensely watchable. this one in particular has the most complete plot with characters who are fearless and show each other mercy. on atmosphere alone the winner could go to maybe four or five of ghibli's other works. i remember watching this one as a kid pretty intensely since it's so violent and the sound effects try to reduce the gore to these cartoonish kinda tom and jerry actions but theres still so much blood. even when i was 10 or 12 i begin to notice that there weren't really characters like <PERSON>'s in american children's media, like role models that had personal resolve but that were also deeply flawed. cartoons here are much more concerned with being sardonic or melodramatic. the part where san baby birds the jerky in the main dude's mouth always gets me",0,"[1.6476448774337769, -1.2187529802322388, -0.9693524241447449, 0.131813183426857, -0.2457742691040039, -0.5583204627037048, 1.21274995803833]"
1,"The Power of the Dog\n\nin isolation there are very strong parts, unfortunately connected together with poorly directed ones that kind of meander without direction. the best scenes are cut too short and show nothing and the badly filmed ones always stretch way too long and show way too much. this film as a whole explores who or what killed <PERSON>. we know, because we've seen every part of the chain that leads to his death. however, where the entire thing fails is that it almost seems like we know more than the movie does","<PERSON>\n\ngoing back and watching these in the original language instead of the goofy english dubs. a lot of people say this is the best one but i'm not really sure. most of them are pretty great. all of them are intensely watchable. this one in particular has the most complete plot with characters who are fearless and show each other mercy. on atmosphere alone the winner could go to maybe four or five of ghibli's other works. i remember watching this one as a kid pretty intensely since it's so violent and the sound effects try to reduce the gore to these cartoonish kinda tom and jerry actions but theres still so much blood. even when i was 10 or 12 i begin to notice that there weren't really characters like <PERSON>'s in american children's media, like role models that had personal resolve but that were also deeply flawed. cartoons here are much more concerned with being sardonic or melodramatic. the part where

In [20]:
all_results_df.layer.value_counts()

layer
0    40
2    40
3    40
4    40
5    40
6    40
1    24
Name: count, dtype: int64

In [23]:
def aggregate_score(list_of_scores):
    scores = []
    for json_str in list_of_scores:
        if json_str == None:
            continue
        try:
            if "\n\n" in json_str:
                json_str = json_str.split("\n")[-1].strip()

            json_str = json_str.replace("Output:", "").strip()
                
                
            json_obj = json.loads(json_str)            
            scores.append(int(json_obj['score']))
        except:
            print(json_str)
            print('==============')
    return round(np.median(scores), 2)

# Aggregating layer score
layer_scores_df = all_results_df.groupby('layer').aggregate({
    'lexical': aggregate_score,
    'syntax': aggregate_score,
    'discourse': aggregate_score,
    'semantic': aggregate_score,
}).reset_index()

In [24]:
layer_scores_df.head(n=7)

,layer,lexical,syntax,discourse,semantic
0,0,1.0,2.0,2.0,1.0
1,1,2.0,3.0,3.0,2.0
2,2,1.0,3.0,2.0,1.0
3,3,2.0,3.0,2.0,2.0
4,4,2.0,3.0,2.0,2.0
5,5,2.0,3.0,2.5,2.0
6,6,2.0,3.0,2.5,2.0
